In [49]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
from google.cloud.bigtable.row_set import RowSet
import google.cloud.bigtable.row_filters as row_filters
import pandas as pd

### Conection

In [13]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID1 = 'p1_gen{}'.format(time.time())
TABLE_ID2 = 'p2_gen{}'.format(time.time())
TABLE_ID3 = 'p1_weather{}'.format(time.time())
TABLE_ID4 = 'p2_weather{}'.format(time.time())
client = bigtable.Client.from_service_account_json('../unbosque-service-account.json', admin = True)
instance = client.instance(  INSTANCE_ID)

In [14]:
p1_gendata = pd.read_csv("../data/Plant_1_Generation_Data.csv")
p1_weatherdata = pd.read_csv("../data/Plant_1_Weather_Sensor_Data.csv")
p2_gendata = pd.read_csv("../data/Plant_2_Generation_Data.csv")
p2_weatherdata = pd.read_csv("../data/Plant_2_Weather_Sensor_Data.csv")

### Creating Tables

In [15]:
"""Creating Tables"""
print('Creating the tables')
table1 = instance.table(TABLE_ID1)
table2 = instance.table(TABLE_ID2)
table3 = instance.table(TABLE_ID3)
table4 = instance.table(TABLE_ID4)


if not table1.exists() and not table2.exists() and not table3.exists() and not table4.exists():
    table1.create()
    table2.create()
    table3.create()
    table4.create()
else:
    print("ERROR: Table already exists")


Creating the Ambiente1649480064.9717906 table


In [18]:
print('Creating all family columns from tables')
p1_power_cfId = 'p1_power'
p1_power_cf = table1.column_family(p1_power_cfId)
p1_power_cf.create()

p2_power_cfId = 'p2_power'
p2_power_cf = table2.column_family(p2_power_cfId)
p2_power_cf.create()

p1_weather_cfId = 'p1_weather'
p1_weather_cf = table3.column_family(p1_weather_cfId)
p1_weather_cf.create()

p2_weatherdata_cfId = 'p2_weather'
p2_weatherdata_cf = table4.column_family(p2_weatherdata_cfId)
p2_weatherdata_cf.create()

Creating all family columns
ID'S: DATE_TIME,PLANT_ID,SOURCE_KEY,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD


### Inserting data into table plant 1 generation data 

In [ ]:
dt = dt.utcnow()
rows = []
json1 = p1_gendata.to_dict(orient='records')
json2 = p2_gendata.to_dict(orient='records')
json3 = p1_weatherdata.to_dict(orient='records')
json4 = p2_weatherdata.to_dict(orient='records')

print('Writing Table 1')
for power in json1:
    row_key = 'plant1#{}'.format(power['SOURCE_KEY']).encode()
    row = table1.direct_row(row_key)
    row.set_cell(p1_power_cfId, 'DC_POWER'.encode(), str(power['POWER']), timestamp = dt)
    row.set_cell(p1_power_cfId, 'AC_POWER'.encode(), str(power['AC_POWER']), timestamp = dt)
    row.set_cell(p1_power_cfId, 'DAILY_YIELD'.encode(), str(power['DAILY_YIELD']), timestamp = dt)
    row.set_cell(p1_power_cfId, 'TOTAL_YIELD'.encode(), str(power['TOTAL_YIELD']), timestamp = dt)
    rows.append(row)
table1.mutate_rows(rows)

print('Writing Table 2')
for power in json2:
    row_key = 'plant2#{}'.format(power['SOURCE_KEY']).encode()
    row = table2.direct_row(row_key)
    row.set_cell(p2_power_cfId, 'DC_POWER'.encode(), str(power['POWER']), timestamp = dt)
    row.set_cell(p2_power_cfId, 'AC_POWER'.encode(), str(power['AC_POWER']), timestamp = dt)
    row.set_cell(p2_power_cfId, 'DAILY_YIELD'.encode(), str(power['DAILY_YIELD']), timestamp = dt)
    row.set_cell(p2_power_cfId, 'TOTAL_YIELD'.encode(), str(power['TOTAL_YIELD']), timestamp = dt)
    rows.append(row)
table2.mutate_rows(rows)

print('Writing Table 3')
for weather in json3:
    row_key = 'plant1#{}'.format(weather['DATE_TIME']).encode()
    row = table3.direct_row(row_key)
    row.set_cell(p1_weather_cfId, 'ambient_temp'.encode(), str(weather['AMBIENT_TEMPERATURE']), timestamp = dt)
    row.set_cell(p1_weather_cfId, 'module_temp'.encode(), str(weather['MODULE_TEMPERATURE']), timestamp = dt)
    row.set_cell(p1_weather_cfId, 'irradiation'.encode(), str(weather['IRRADIATION']), timestamp = dt)
    rows.append(row)
table3.mutate_rows(rows)
  
print('Writing Table 4')
for weather in json4:
    row_key = 'plant2#{}'.format(weather['DATE_TIME']).encode()
    row = table4.direct_row(row_key)
    row.set_cell(p2_weatherdata_cfId, 'ambient_temp'.encode(), str(weather['AMBIENT_TEMPERATURE']), timestamp = dt)
    row.set_cell(p2_weatherdata_cfId, 'module_temp'.encode(), str(weather['MODULE_TEMPERATURE']), timestamp = dt)
    row.set_cell(p2_weatherdata_cfId, 'irradiation'.encode(), str(weather['IRRADIATION']), timestamp = dt)
    rows.append(row)
